# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
#from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=1024, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=1024, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        one_conv_kernel_size=103
        self.linear = nn.Conv1d(1, 1, one_conv_kernel_size, stride=103,padding=3,groups=1,dilation=1,bias=True) # nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        #print("size1:",out.shape)
        bs=out.shape[0]
        out=out.view(bs,1,1024)
        out = self.linear(out)
        #print("size1:",out.shape)
        out=out.view(bs,10) # 10 class
        return out

In [4]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
if torch.cuda.is_available():
    model=MobileNet(10).cuda()
else:
    model=MobileNet(10)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [5]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            if batch_idx % 4*show_step==0:
                print(torch.abs(model.layers[12].conv2.weight).sum())
                print(model.layers[12].conv2.weight.shape)
                print(model.layers[12].conv2.weight)
                
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [6]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [7]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [8]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [9]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
#writer = SummaryWriter()
#writer.export_scalars_to_json("./all_scalars.json")

#writer.close()

#---------------------------- Test ------------------------------
test(epoch)

tensor(17876.0449, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.6158e-02]],

         [[-1.5721e-02]],

         [[ 5.0717e-03]],

         ...,

         [[ 3.0422e-02]],

         [[-1.4717e-02]],

         [[-1.7136e-02]]],


        [[[ 1.0009e-02]],

         [[-1.3958e-02]],

         [[ 1.9527e-02]],

         ...,

         [[-1.0740e-02]],

         [[ 5.4821e-03]],

         [[ 4.0501e-02]]],


        [[[-1.9136e-02]],

         [[-1.3957e-02]],

         [[-9.2750e-03]],

         ...,

         [[ 2.5767e-02]],

         [[-1.8304e-02]],

         [[-1.3609e-03]]],


        ...,


        [[[-7.8839e-03]],

         [[-1.4723e-02]],

         [[ 5.7652e-03]],

         ...,

         [[ 9.9766e-03]],

         [[ 2.6063e-02]],

         [[-2.1302e-02]]],


        [[[-3.5930e-02]],

         [[ 2.0733e-02]],

         [[ 3.8523e-03]],

         ...,

         [[ 3.3510e-03]],

         [[-1.1828e-02]],

         [[-1.5236e-02]]],


 

Train Epoch: 2 [10240/50000 (23%)]	Loss: 1.210823, Accuracy: 55.18
tensor(61245.9102, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 7.8230e-03]],

         [[-6.7339e-02]],

         [[ 7.1538e-02]],

         ...,

         [[-9.5010e-03]],

         [[ 4.8931e-02]],

         [[-1.9042e-02]]],


        [[[-1.3972e-01]],

         [[-9.0390e-02]],

         [[ 7.1027e-02]],

         ...,

         [[ 7.7392e-03]],

         [[ 7.1116e-02]],

         [[ 8.9553e-02]]],


        [[[-8.0375e-03]],

         [[ 4.2714e-02]],

         [[ 8.4415e-02]],

         ...,

         [[-2.2290e-02]],

         [[ 3.3986e-02]],

         [[ 2.2265e-02]]],


        ...,


        [[[ 3.7263e-02]],

         [[ 2.9946e-02]],

         [[ 6.0866e-02]],

         ...,

         [[-2.7122e-02]],

         [[ 5.9388e-02]],

         [[-9.7219e-02]]],


        [[[-1.6139e-01]],

         [[-2.7620e-02]],

         [[-1.4653e-02]],

         ...,

         [[-3.359

Train Epoch: 4 [30720/50000 (68%)]	Loss: 0.842954, Accuracy: 69.82
tensor(67879.8281, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 2.5952e-02]],

         [[-7.5499e-02]],

         [[ 6.1527e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 4.8477e-02]],

         [[-4.1075e-02]]],


        [[[-1.3588e-01]],

         [[-7.6239e-02]],

         [[ 1.7086e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 7.3945e-02]],

         [[ 7.9590e-02]]],


        [[[ 2.2563e-02]],

         [[ 3.6833e-02]],

         [[ 6.6583e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.1815e-02]],

         [[-1.0960e-02]]],


        ...,


        [[[ 3.9127e-02]],

         [[ 3.0121e-02]],

         [[ 5.8027e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 6.6757e-02]],

         [[-9.9915e-02]]],


        [[[-1.3185e-01]],

         [[-4.6767e-02]],

         [[-2.8793e-03]],

         ...,

         [[-2.850


Validation set: Average loss: 0.9740, Accuracy: 3431/5000 (68.00%)

the time of this epoch:[18.528116941452026 s]
tensor(72697., device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 2.5289e-02]],

         [[-7.8052e-02]],

         [[ 1.0648e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 4.6139e-02]],

         [[-4.6752e-02]]],


        [[[-1.4218e-01]],

         [[-8.3051e-02]],

         [[-2.4845e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 7.5472e-02]],

         [[ 6.8449e-02]]],


        [[[ 2.5620e-02]],

         [[ 3.1147e-02]],

         [[ 1.1773e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.1106e-02]],

         [[-2.6148e-02]]],


        ...,


        [[[ 3.8799e-02]],

         [[ 2.7117e-02]],

         [[ 3.9643e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 6.0935e-02]],

         [[-1.0239e-01]]],


        [[[-6.4372e-02]],

         [[-5.6041e-02]],

         [[-4.0

Train Epoch: 9 [10240/50000 (23%)]	Loss: 0.519471, Accuracy: 83.59
tensor(77331.2188, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 3.4587e-02]],

         [[-6.8436e-02]],

         [[ 2.3673e-03]],

         ...,

         [[-9.2328e-03]],

         [[ 5.1155e-02]],

         [[-2.2094e-02]]],


        [[[-1.4485e-01]],

         [[-7.4350e-02]],

         [[-4.2985e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 8.3429e-02]],

         [[ 7.6166e-02]]],


        [[[ 4.1700e-02]],

         [[ 4.6058e-02]],

         [[-2.8633e-03]],

         ...,

         [[-2.1528e-02]],

         [[ 3.1788e-02]],

         [[ 3.2468e-03]]],


        ...,


        [[[ 3.0493e-02]],

         [[ 2.1791e-02]],

         [[ 4.4946e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 6.0241e-02]],

         [[-9.5417e-02]]],


        [[[-9.6155e-03]],

         [[-7.2936e-02]],

         [[-5.2978e-02]],

         ...,

         [[-2.850

...


Validation set: Average loss: 0.5222, Accuracy: 4444/5000 (88.00%)

the time of this epoch:[18.535001039505005 s]
tensor(86403.4453, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 4.5810e-02]],

         [[-6.7749e-02]],

         [[ 7.5332e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 6.8850e-02]],

         [[ 7.1542e-02]]],


        [[[-1.2431e-01]],

         [[-1.0502e-01]],

         [[-2.2053e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 1.0344e-01]],

         [[ 1.2161e-01]]],


        [[[ 5.8738e-02]],

         [[ 4.1746e-02]],

         [[ 6.9766e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.9529e-02]],

         [[ 9.5681e-02]]],


        ...,


        [[[ 9.0627e-03]],

         [[ 1.7444e-02]],

         [[ 1.0545e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 5.4923e-02]],

         [[-7.6769e-02]]],


        [[[ 7.1898e-02]],

         [[-8.3406e-02]],

         [[

Train Epoch: 51 [10240/50000 (23%)]	Loss: 0.054295, Accuracy: 98.05
tensor(86412.2578, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 4.5234e-02]],

         [[-6.8578e-02]],

         [[ 7.5821e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 6.7999e-02]],

         [[ 7.0954e-02]]],


        [[[-1.2311e-01]],

         [[-1.0813e-01]],

         [[-2.0119e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 1.0127e-01]],

         [[ 1.2047e-01]]],


        [[[ 5.8077e-02]],

         [[ 4.0513e-02]],

         [[ 7.0317e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.9156e-02]],

         [[ 9.5047e-02]]],


        ...,


        [[[ 1.1614e-02]],

         [[ 1.6842e-02]],

         [[ 1.4574e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 5.5338e-02]],

         [[-8.2618e-02]]],


        [[[ 6.8435e-02]],

         [[-8.2411e-02]],

         [[-6.9444e-02]],

         ...,

         [[-2.85

Train Epoch: 53 [30720/50000 (68%)]	Loss: 0.034038, Accuracy: 98.83
tensor(86419.6016, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 4.6227e-02]],

         [[-6.8336e-02]],

         [[ 7.7252e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 6.9086e-02]],

         [[ 7.1151e-02]]],


        [[[-1.2228e-01]],

         [[-1.0745e-01]],

         [[-1.8069e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 1.0341e-01]],

         [[ 1.2049e-01]]],


        [[[ 5.9240e-02]],

         [[ 4.1033e-02]],

         [[ 7.1796e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.9638e-02]],

         [[ 9.5334e-02]]],


        ...,


        [[[ 1.2259e-02]],

         [[ 1.6637e-02]],

         [[ 1.2357e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 5.4962e-02]],

         [[-8.2021e-02]]],


        [[[ 6.7503e-02]],

         [[-8.2505e-02]],

         [[-6.8424e-02]],

         ...,

         [[-2.85


Validation set: Average loss: 0.5520, Accuracy: 4444/5000 (88.00%)

the time of this epoch:[18.469688415527344 s]
tensor(86418.7812, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 4.5373e-02]],

         [[-6.9156e-02]],

         [[ 7.7111e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 6.9273e-02]],

         [[ 7.0454e-02]]],


        [[[-1.2328e-01]],

         [[-1.0936e-01]],

         [[-1.7608e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 1.0369e-01]],

         [[ 1.1942e-01]]],


        [[[ 5.8264e-02]],

         [[ 3.9670e-02]],

         [[ 7.1719e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.9753e-02]],

         [[ 9.4499e-02]]],


        ...,


        [[[ 1.2927e-02]],

         [[ 1.6932e-02]],

         [[ 1.0546e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 5.4231e-02]],

         [[-7.9561e-02]]],


        [[[ 6.8184e-02]],

         [[-8.3867e-02]],

         [[

Train Epoch: 58 [10240/50000 (23%)]	Loss: 0.028189, Accuracy: 99.12
tensor(86407.1016, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 4.5254e-02]],

         [[-6.8466e-02]],

         [[ 7.4596e-02]],

         ...,

         [[-9.2328e-03]],

         [[ 6.8923e-02]],

         [[ 7.0814e-02]]],


        [[[-1.2376e-01]],

         [[-1.0788e-01]],

         [[-2.1261e-02]],

         ...,

         [[ 6.7863e-03]],

         [[ 1.0276e-01]],

         [[ 1.2009e-01]]],


        [[[ 5.8148e-02]],

         [[ 4.0498e-02]],

         [[ 6.9092e-02]],

         ...,

         [[-2.1528e-02]],

         [[ 3.9595e-02]],

         [[ 9.4841e-02]]],


        ...,


        [[[ 1.3034e-02]],

         [[ 1.6503e-02]],

         [[ 1.4372e-02]],

         ...,

         [[-2.5991e-02]],

         [[ 5.5633e-02]],

         [[-7.9979e-02]]],


        [[[ 6.8067e-02]],

         [[-8.3669e-02]],

         [[-7.0316e-02]],

         ...,

         [[-2.85

# Step 3: Test

In [12]:
test(epoch)


Test set: Average loss: 0.4542, Accuracy: 8932/10000 (89.32%)

